# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 2 [10 баллов] до 23:59 22.03.2018. Предсказание цены акции по экономическим новостям


В этом домашнем задании вы попытаетесь предсказать рост цены акции компании Газпром по новостям о компании. Домашнее задание состоит из трех частей:
1. Предварительная обработка текстов и эксплоративный анализ
2. Baseline алгоритм
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



Входные данные:
* Новости о компании "Газпром", начиная с 2010 года
* Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
    * цена открытия (Open)
    * цена закрытия (ClosingPrice)
    * максимальная цена за день (DailyHigh)
    * минимальная цена за день (DailyLow) 
    * объем бумаг (VolumePcs)


Задание делали: Захар Максименко, Анастасия Максимовская, Константин Ваниев (все - ИАД-4).

In [1]:
import nltk
import pandas as pd
import re
import pymorphy2
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('texts.csv')
df.head()

,date,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [2]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all['ClosingPrice'] = pr_all['ClosingPrice'].apply(lambda x: float(x.replace(',', '.')))
pr_all['Open'] = pr_all['Open'].astype(str).apply(lambda x: float(x.replace(',', '.')))
pr_all['DailyHigh'] = pr_all['DailyHigh'].astype(str).apply(lambda x: float(x.replace(',', '.')))
pr_all['DailyLow'] = pr_all['DailyLow'].astype(str).apply(lambda x: float(x.replace(',', '.')))
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,08.12.2017,133.43,132.60,133.90,132.00,16037970
1,07.12.2017,133.70,133.02,133.87,132.81,18198430
2,06.12.2017,133.33,134.00,134.29,132.91,14641730
3,05.12.2017,133.48,133.65,133.99,132.78,12684800
4,04.12.2017,133.01,133.77,134.00,131.93,17818980


## Часть 1. Вводная

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* Ско
* О каких санкциях пишут в статьях?

In [3]:
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
lemmed_texts = []
df['text'] = df['text'].str.lower()
reg = re.compile('[a-zа-я\-]+')
for i in range(len(df['text'])):
    l1 = reg.findall( df['text'][i])
    l3 = [morph.parse(token)[0].normal_form for token in l1 if not token in stopwords.words('russian')] 
    lemmed_texts.append(l3)

В ячейке выше мы токенизировали каждый документ в нашем маленьком корпусе. Также каждое слово приведено к строчным буквам, небуквенные символы (кроме дефиса) убраны, а слова приведены к начальной форме путем лемматизации.

In [4]:
non_tokenized = []
for i in lemmed_texts:
    j = (' ').join(i)
    non_tokenized.append(j)

На всякий случай сделаем лемматизированную версию каждой новости без токенизации.

Теперь дадим ответы на вопросы о том, кого чаще упоминают. "Частота упоминания", на наш взгляд, имеет два значения - количество уникальных упоминаний в документах корпуса (во скольких документах появится Х) и количество упоминаний вообще (сколько раз Х встречается во всех документах). Уникальные упоминания посчитаны ниже.

In [5]:
from collections import Counter

def who_wins(first, second):
    t1 = 0
    t2 = 0
    for i in non_tokenized:
        if (i.find(first) != -1):
            t1 += 1
        if (i.find(second) != -1):
            t2 += 1
    print('Упоминания: ', first, "-", t1, ',', second, '-', t2)

In [6]:
who_wins('путин', 'миллер')

Упоминания:  путин - 61 , миллер - 102


In [7]:
who_wins('владимир путин', 'алексей миллер')

Упоминания:  владимир путин - 59 , алексей миллер - 102


Можно заметить, что добавление имени практически не поменяло количество уникальных упоминаний - почти всегда фамилия этих людей идет вместе с именем. Миллер появляется в большем числе документов.

In [8]:
who_wins('северный поток', 'турецкий поток')

Упоминания:  северный поток - 12 , турецкий поток - 34


In [9]:
who_wins('nord stream', 'south stream')

Упоминания:  nord stream - 63 , south stream - 38


Если следовать "букве" задания, то можно сделать вывод, что северный поток появляется в документах чаще. Однако если сравнить количество уникальных упоминаний официальных международных наименований этих проектов, то становится очевидно, что северный поток обсуждается чаще.

In [10]:
from nltk.text import Text
for i in range(len(lemmed_texts)):
    if (non_tokenized[i].find('санкция') != -1):
        Text(lemmed_texts[i]).concordance('санкция')

Displaying 1 of 1 matches:
                                    санкция ставить угроза зарубежный нефтяной 
Displaying 2 of 2 matches:
 участок интересоваться shell из-за санкция иностранец год шельф россия пока ид
 участок интересоваться shell из-за санкция иностранец год шельф россия пока ид
Displaying 2 of 2 matches:
компания рф наблюдаться число из-за санкция эксперт полагать сила опыт организа
компания рф наблюдаться число из-за санкция эксперт полагать сила опыт организа
Displaying 1 of 1 matches:
изингодатель почувствовать западный санкция ъ рассказать гендиректор газпромбан
Displaying 1 of 1 matches:
рдить правомерность ввести совет ес санкция отношение роснефть брюссель запрети
Displaying 2 of 2 matches:
ение дело недовольный грозить банка санкция территория украина крупный российск
ение дело недовольный грозить банка санкция территория украина
Displaying 1 of 1 matches:
жно-киринский месторождение попасть санкция мнение эксперт пока речь идти полит
Displaying 1 of 1 matches:
е

С помощью вывода контекста для слова "санкция" в каждом документе, где встречается слово, можно понять, о каких санкциях идёт речь [хоть в лемматизированном тексте это не совсем обычно выглядит :) ]. Можно заметить, что речь идёт о санкциях США, санкциях, который вводит совет ЕС, санкциях Украины, западных санкциях. До периода российских политических проблем говорится о санкциях, наложенных на Кубу, а также о "штрафных санкциях", которые вообще совсем не политические. Много санкций хороших и разных!

In [11]:
prices = []
for i in range(len(df['date'])):
    if len(np.where(pr_all['Date'] == df['date'][i])[0]) > 0:
        prices.append(pr_all['ClosingPrice'][np.where(pr_all['Date'] == df['date'][i])[0][0]])
    else:
        prices.append('nan')

Чтобы строить какие-то корреляции, нужно четко сопоставить тексты и цены. Не для каждого текста есть цена, не для каждой цены есть текст. В ячейке выше мы создаем массив цены в день написания каждого доступного нам текста. Если цены нет, то мы ставим nan. Далее мы отбираем тексты, для которых есть цены, и из них формируем массивы значений. Первый с количеством упоминаний миллера (как мы выяснили в прошлом задании, между "миллером" и "алексеем миллером" разницы в количестве уникальных упоминаний практически нет, а вот по фамилии в рамках одного текста его могут назвать чаще в случае повторного упоминания). Второй с длинами текстов. Далее мы аналогичным образом отбираем цены, для которых есть тексты и считаем корреляцию.

In [12]:
texts_no_nan = []
for i in np.where(np.array(prices) != 'nan')[0]:
    texts_no_nan.append(lemmed_texts[i])

In [13]:
miller_texts = []
for i in texts_no_nan:
    miller_texts.append(i.count('миллер'))

In [14]:
len_texts = []
for i in texts_no_nan:
    len_texts.append(len(i))

In [15]:
prices_corr = np.array(prices)[np.where(np.array(prices) != 'nan')].astype(float)

In [16]:
np.corrcoef(miller_texts,prices_corr)[0][1]

0.010635076467578013

In [17]:
np.corrcoef(len_texts,prices_corr)[0][1]

0.014823415198165359

Корреляции нет. На наш взгляд, это связано с наличием множества других факторов, оказывающих влияние на цену, а также с большим временным разбросом новостей. То есть даже если бы упоминание Миллера имело бы связь с динамикой цены в конкретный день, существует множество факторов, из-за которых динамика может быть совершенно другой. Также связь с динамикой цены в конкретный день (если бы она была) не всегда означает высокую корреляцию - средний уровень цены акции в разные годы существенно отличается и по очевидным причинам зависит не от частоты упоминания Миллера в новостях.

## Часть 2. Классификационная
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
* составьте бучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

In [18]:
dynamics = np.zeros(len(df))
for i in range(len(df)):
    if len(pr_all[pr_all['Date'] == df['date'][i]]) != 0:
        index = np.where(pr_all['Date'] == df['date'][i])[0][0]
        if index == len(pr_all) - 1:
            break
        price = pr_all['ClosingPrice'][index]
        prev_price = pr_all['ClosingPrice'][index + 1]
        if type(price) == np.float64 and type(prev_price) == np.float64:
            if price > prev_price:
                dynamics[i] = 1
        else:
            dynamics[i] = 'nan'
    else:
        dynamics[i] = 'nan'

В первую очередь нужно создать бинарный признак. Описание алгоритма: Если для текста есть соответствующая дата в таблице с ценами и если эта дата - число, а не 'nan', то при условии, что цена за день с новостью больше предыдущей, то мы заносим в бинарный признак единицу. Если обратное - 0. Если нарушено одно из начальных условий - 'nan'. Далее мы убираем тексты и цены, для которых соответствия не нашлось.

In [19]:
dynamics_match = []
texts_match = []
for i in range(len(dynamics)):
    if not np.isnan(dynamics[i]):
        dynamics_match.append(dynamics[i])
        texts_match.append(non_tokenized[i])

Попробуем по-тупому. CountVectorizer просто сделает нам мешок слов, Tfidfvectorizer сделает мешок и взвесит слова в нём.

In [20]:
dfstem = pd.DataFrame(data=dynamics_match)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(texts_match).toarray()
df_cv = pd.DataFrame(data=x, columns=v.get_feature_names())

Очевидно, что львиная доля работы - отбор признаков. В нашем случае - отбор слов, которые могут быть важными признаками для нашей модели. Критерием важности для модели сделаем сумму значений tf-idf для одного слова во всех документах. 

In [22]:
counter = []
for i in range(df_cv.shape[1]):
    counter.append(np.sum(df_cv.iloc[ :, [i]])[0])

Идея следующая: мы можем отобрать слова и проверить их важность для модели, зафитив целевую переменную с набором данных, состоящим из одного признака - значений tf-idf для какого-либо слова. Далее все слова, показавшие точность большую, чем некий уровень отсечения, определяются как важные признаки и вносятся в итоговый набор данных, на котором в итоге фитится основная модель. Мы получаем две переменные, от которых может зависеть результат - уровень суммы tf-idf для каждого слова, ниже которого слова мы брать не будем (оно может быть очень распространенным во всех документах и не иметь значения как различительный фактор, или напротив, оно может попасться 1-2 раза и его результат на accuracy может быть нерепрезентативным). Второй показатель - уровень отсечения accuracy, слова с уровнем ниже заданного не попадут в итоговую модель. Ниже представлены функция, реализующая этот алгоритм, а также способ отбора нужных значений переменных.

Range подставляемых в обе переменные значений выявлен путём trial-and-error и сужением изначального range.

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
def feature_selector(tfidf_sum, threshold):
    word_filter = []
    y_train = dfstem[258:]
    y_test = dfstem[:258]
    for j in range(len(df_cv.columns)):
        if counter[j] > tfidf_sum:
            X_train = df_cv.filter([df_cv.columns[j]], axis = 1)[258:]
            X_test = df_cv.filter([df_cv.columns[j]], axis = 1)[:258]
            TradingGuru = GaussianNB()
            TradingGuru.fit(X_train, y_train)
            y_pred = TradingGuru.predict(X_test)
            result = accuracy_score(y_test, y_pred)
            if result > threshold:
                word_filter.append(df_cv.columns[j])
    X_train, y_train = df_cv.filter(word_filter, axis = 1)[258:], dfstem[258:]
    X_test, y_test = df_cv.filter(word_filter, axis = 1)[:258], dfstem[:258]
    TradingGuru = GaussianNB()
    TradingGuru.fit(X_train, y_train)
    y_pred = TradingGuru.predict(X_test)
    result = accuracy_score(y_test, y_pred)
    return result

In [24]:
sums, thresholds, results = [], [], []
for i in np.linspace(0.5,2,4):
    for j in np.linspace(0.54, 0.57, 4):
        sums.append(i)
        thresholds.append(j)
        results.append(feature_selector(i, j))

C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:432: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)


In [25]:
print('optimal tf-idf sum threshold:', sums[np.where(results == np.max(results))[0][0]])
print('optimal accuracy threshold:', thresholds[np.where(results == np.max(results))[0][0]])
print('accuracy score:', np.max(results))

optimal tf-idf sum threshold: 1.0
optimal accuracy threshold: 0.56
accuracy score: 0.751937984496


Точность составила около 76%. Чтобы понять, как бы было, если бы использовали простой CountVectorizer, сравним его результаты, полученные таким же образом, с полученными выше.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(token_pattern=r"\b\w+\b")
cv_matrix = count.fit_transform(texts_match).toarray()
df_cv = pd.DataFrame(data=cv_matrix, columns=count.get_feature_names())

In [27]:
counter = []
for i in range(df_cv.shape[1]):
    counter.append(np.sum(df_cv.iloc[ :, [i]])[0])

In [28]:
def feature_selector(tfidf_sum, threshold):
    word_filter = []
    y_train = dfstem[258:]
    y_test = dfstem[:258]
    for j in range(len(df_cv.columns)):
        if counter[j] > tfidf_sum:
            X_train = df_cv.filter([df_cv.columns[j]], axis = 1)[258:]
            X_test = df_cv.filter([df_cv.columns[j]], axis = 1)[:258]
            TradingGuru = GaussianNB()
            TradingGuru.fit(X_train, y_train)
            y_pred = TradingGuru.predict(X_test)
            result = accuracy_score(y_test, y_pred)
            if result > threshold:
                word_filter.append(df_cv.columns[j])
    X_train = df_cv.filter(word_filter, axis = 1)[258:]
    X_test = df_cv.filter(word_filter, axis = 1)[:258]
    TradingGuru = GaussianNB()
    TradingGuru.fit(X_train, y_train)
    y_pred = TradingGuru.predict(X_test)
    result = accuracy_score(y_test, y_pred)
    return result

In [29]:
sums, thresholds, results = [], [], []
for i in np.linspace(0,4,5):
    for j in np.linspace(0.55, 0.56, 2):
        sums.append(i)
        thresholds.append(j)
        results.append(feature_selector(i, j))
        print(i,j)

C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:432: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)


0.0 0.55
0.0 0.56
1.0 0.55
1.0 0.56
2.0 0.55
2.0 0.56
3.0 0.55
3.0 0.56
4.0 0.55
4.0 0.56


In [30]:
print('optimal countvect sum threshold:', sums[np.where(results == np.max(results))[0][0]])
print('optimal accuracy threshold:', thresholds[np.where(results == np.max(results))[0][0]])
print('accuracy score:', np.max(results))

optimal countvect sum threshold: 0.0
optimal accuracy threshold: 0.55
accuracy score: 0.763565891473


Точность совпадает, кроме того, CountVectorizer не потребовал никакой "точки отсечения" по количеству упоминаний слов во всех документах.

## Часть 3. Творческая
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по эмбеддингам глаголов (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

### Попробуем проделать первый предложенный вариант с использованием только существительных

In [33]:
nltk.download('averaged_perceptron_tagger_ru')
from nltk import pos_tag
nouns=[]
for i in range(len(texts_match)):
    l1 = reg.findall(texts_match[i])
    l3 = [w for w , pos in pos_tag(l1, lang='rus') if (pos == 'S')]   # находим существительные
    nouns.append(l3)

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


In [34]:
texts_n = []
for i in nouns:
    j = (' ').join(i)
    texts_n.append(j)        # делаем "тексты" только с существительными

In [35]:
x_n = v.fit_transform(texts_n).toarray()
df_cv_n = pd.DataFrame(data=x_n, columns=v.get_feature_names())

In [36]:
counter_n = []
for i in range(df_cv_n.shape[1]):
    counter_n.append(np.sum(df_cv_n.iloc[ :, [i]])[0])

In [37]:
def feature_selector_n(tfidf_sum, threshold):
    word_filter = []
    y_train = dfstem[258:]
    y_test = dfstem[:258]
    for j in range(len(df_cv_n.columns)):
        if counter_n[j] > tfidf_sum:
            X_train = df_cv_n.filter([df_cv_n.columns[j]], axis = 1)[258:]
            X_test = df_cv_n.filter([df_cv_n.columns[j]], axis = 1)[:258]
            TradingGuru = GaussianNB()
            TradingGuru.fit(X_train, y_train)
            y_pred = TradingGuru.predict(X_test)
            result = accuracy_score(y_test, y_pred)
            if result > threshold:
                word_filter.append(df_cv_n.columns[j])
    X_train, y_train = df_cv_n.filter(word_filter, axis = 1)[258:], dfstem[258:]
    X_test, y_test = df_cv_n.filter(word_filter, axis = 1)[:258], dfstem[:258]
    TradingGuru = GaussianNB()
    TradingGuru.fit(X_train, y_train)
    y_pred = TradingGuru.predict(X_test)
    result = accuracy_score(y_test, y_pred)
    return result

In [38]:
sums, thresholds, results = [], [], []
for i in np.linspace(0.5,2,4):
    for j in np.linspace(0.54, 0.57, 4):
        sums.append(i)
        thresholds.append(j)
        results.append(feature_selector_n(i, j))

C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:432: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)


In [39]:
print('optimal tf-idf sum threshold:', sums[np.where(results == np.max(results))[0][0]])
print('optimal accuracy threshold:', thresholds[np.where(results == np.max(results))[0][0]])
print('accuracy score:', np.max(results))

optimal tf-idf sum threshold: 1.5
optimal accuracy threshold: 0.56
accuracy score: 0.68992248062


In [40]:
cv_matrix_n = count.fit_transform(texts_n).toarray()
df_cv_n = pd.DataFrame(data=cv_matrix_n, columns=count.get_feature_names())

In [41]:
counter_n = []
for i in range(df_cv_n.shape[1]):
    counter_n.append(np.sum(df_cv_n.iloc[ :, [i]])[0])

In [42]:
def feature_selector_n(tfidf_sum, threshold):
    word_filter = []
    y_train = dfstem[258:]
    y_test = dfstem[:258]
    for j in range(len(df_cv_n.columns)):
        if counter_n[j] > tfidf_sum:
            X_train = df_cv_n.filter([df_cv_n.columns[j]], axis = 1)[258:]
            X_test = df_cv_n.filter([df_cv_n.columns[j]], axis = 1)[:258]
            TradingGuru = GaussianNB()
            TradingGuru.fit(X_train, y_train)
            y_pred = TradingGuru.predict(X_test)
            result = accuracy_score(y_test, y_pred)
            if result > threshold:
                word_filter.append(df_cv_n.columns[j])
    X_train = df_cv_n.filter(word_filter, axis = 1)[258:]
    X_test = df_cv_n.filter(word_filter, axis = 1)[:258]
    TradingGuru = GaussianNB()
    TradingGuru.fit(X_train, y_train)
    y_pred = TradingGuru.predict(X_test)
    result = accuracy_score(y_test, y_pred)
    return result

In [43]:
sums, thresholds, results = [], [], []
for i in np.linspace(0,4,5):
    for j in np.linspace(0.55, 0.56, 2):
        sums.append(i)
        thresholds.append(j)
        results.append(feature_selector_n(i, j))
        print(i,j)

C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:432: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:434: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)


0.0 0.55
0.0 0.56
1.0 0.55
1.0 0.56
2.0 0.55
2.0 0.56
3.0 0.55
3.0 0.56
4.0 0.55
4.0 0.56


In [44]:
print('optimal countvect sum threshold:', sums[np.where(results == np.max(results))[0][0]])
print('optimal accuracy threshold:', thresholds[np.where(results == np.max(results))[0][0]])
print('accuracy score:', np.max(results))

optimal countvect sum threshold: 0.0
optimal accuracy threshold: 0.55
accuracy score: 0.682170542636


#### К сожалению, в результате данного способа точность не повысилась, а понизилась. Возможно, слова других частей речи также играли роль в правильной классификации

Кроме того, можно в целом сказать, что предложенный во второй части алгоритм сам по себе является улучшением самого простого бейзлайна, т.к. мы предусмотрели такие параметры как accuracy threshold, tf-idf sum threshold, и отбирали признаки уже в самом бейзлайне, поэтому проведенный выше метод отбора существительных и не дал положительного результата

Как показало исследование, если взять самый незамысловатый бейзлайн, который просто после tf-idf преобразования будет проводить классификацию, у него будет намного ниже точность (0.47), а при отборе только существительных она ненамного, но повысится! Продемонстрируем это ниже:

In [45]:
v = TfidfVectorizer()
x = v.fit_transform(texts_match).toarray()
df_cv = pd.DataFrame(data=x, columns=v.get_feature_names())

X_train, y_train = df_cv.iloc[258:], dfstem[258:]
X_test, y_test = df_cv.iloc[:258], dfstem[:258]
TradingGuru = GaussianNB()
TradingGuru.fit(X_train, y_train)
y_pred = TradingGuru.predict(X_test)
print(accuracy_score(y_test, y_pred))

C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.472868217054


In [46]:
nouns=[]
for i in range(len(texts_match)):
    l1 = reg.findall(texts_match[i])
    l3 = [w for w , pos in pos_tag(l1, lang='rus') if (pos == 'S')]   # находим существительные
    nouns.append(l3)

texts_n = []
for i in nouns:
    j = (' ').join(i)
    texts_n.append(j)        # делаем "тексты" только с существительными
    
x_n = v.fit_transform(texts_n).toarray()
df_cv_n = pd.DataFrame(data=x_n, columns=v.get_feature_names())

In [47]:
X_train_n, y_train_n = df_cv_n.iloc[258:], dfstem[258:]
X_test_n, y_test_n = df_cv_n.iloc[:258], dfstem[:258]
TradingGuru = GaussianNB()
TradingGuru.fit(X_train_n, y_train_n)
y_pred_n = TradingGuru.predict(X_test_n)
print(accuracy_score(y_test_n, y_pred_n))

C:\Users\maxim\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.492248062016


## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 22.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 2: Таисия Глушкова (email: glushkovato@gmail.com, telegram: @glushkovato).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

